In [1]:
from cvxopt.solvers import qp
from cvxopt.base import matrix

import numpy, pylab, random, math


# Why do we care about having a large margin?
Lines that are too close to one class or another class (the positives or negatives), means that you believe in your training data too much. You don't want to believe the data too much, even though its all you have, otherwise you are *overfitting*. Lines that are too close to one of the classes are more likely to overfit. 

Functions which overfit are more complex, so it is interesting to note that the lines which are too close to a particular class are more complex.

This is a good literal interpretation of the words "over" and "fit."


# Lines and Hyperplanes
The equation for a line is relatively straightforward:
y = mx+b

The equation for a hyperplane looks like this:
y = w(transpose)x + b

Heres some definitions for this hyperplane equation:
* y: *classification label. *The output which indicates whether you are in the positive or negative class
* w: parameters for the plane
* b: also another parameter, moves it out of the origin

# Kernel Function

**Task:** Implement a suitable kernel function

## Linear Kernel

This is really just a dot product of two vectors (plus one).

K(vec_x, vec_y) = vec_x_transpose * vec_y + 1

In [1]:
# Mister Global
POWER = 2
RADIAL_OMEGA = 1
SIGMOID_DELTA = 1
SIGMOID_K = 0.5

In [3]:
def linear_kernel(vec_x, vec_y):
    # Take the dot product and then add one
    return numpy.dot(vec_x, vec_y) + 1
    

In [4]:
#Class from Jokull (thanks bro)
class kernels:
   def linear(x, y):
       xTrans = numpy.array(x).transpose()
       return xTrans.dot(y) + 1

   def polynomial(x, y):
       return math.pow(kernels.linear(x,y), POWER)

   def radial(x, y):
       xTrans = numpy.array(x).transpose()
       xy  = numpy.subtract(xTrans, y)
       normXY = numpy.linalg.norm(xy)
       return math.exp( -math.pow(normXY,2)/(2*math.pow(RADIAL_OMEGA,2)))

   def sigmoid(x, y):
       xTrans = numpy.array(x).transpose()
       return numpy.tanh(SIGMOID_K*xTrans.dot(y) - SIGMOID_DELTA)

In [5]:
vec_x = [1,2,3]
vec_y = [1,2,3]
print(linear_kernel(vec_x,vec_y))

15


# Build a P matrix


In [6]:
def build_p_matrix(samples, kernel_fun):
    matrix = []
    x_i = []
    x_j = []
    for i in range(len(samples)):
        row = []
        for j in range(len(samples)):
            P_element = samples[i][2] * samples[j][2] * kernel_fun([samples[i][0], samples[i][1]], 
                                                                   [samples[j][0], samples[j][1]])
            row.append(P_element)
        matrix.append(row)
    return matrix



# Generate Test Data

In [7]:
classA = [(random.normalvariate(-1.5, 1),
          random.normalvariate(0.5, 1),
          1.0)
          for i in range(5)] + \
        [(random.normalvariate(1.5, 1),
         random.normalvariate(0.5, 1),
         1.0)
        for i in range(5)]
    
classB = [(random.normalvariate(0.0, 0.5),
          random.normalvariate(-0.5, 0.5),
          -1.0)
         for i in range(10)]

data = classA + classB
random.shuffle(data)
#Jokull 
data = [(-0.4817775866423481, -0.676196126111814, -1.0), (-1.3746653598765421, -1.2335653503955357, 1.0), (-0.04952991285713326, -0.477259823574604, -1.0), (-2.435302270702021, 0.5791628345129953, 1.0), (-0.23207345925812842, -0.829241385869637, -1.0), (-1.746084907233614, 0.9006137892797217, 1.0), (-0.3698769551877943, 1.9746639924223914, 1.0), (1.3807418306962262, 0.8160567394627515, 1.0), (0.6481506000006643, -0.2993683817580585, -1.0), (0.006408247677090779, -0.711255503088516, -1.0), (-0.3556075093525846, 0.6450331217049499, -1.0), (0.9314091400315766, -0.7871051525340638, -1.0), (0.86816793401546, -0.5298525064359068, 1.0), (-0.04468918689694391, -0.5181337266696356, -1.0), (1.2636537068237677, 1.1427041812767662, 1.0), (0.09043582719932093, 0.04380816810541566, -1.0), (0.012425436725393221, -0.6643420601684578, -1.0), (-2.0485964643706644, 0.6991611363135597, 1.0), (-2.8139016711910467, 0.17553854976255767, 1.0), (1.6300789364960653, 0.3280724548177084, 1.0)]
N = len(data)

In [8]:
pylab.hold(True)
pylab.plot([p[0] for p in classA],
          [p[1] for p in classA],
          'bo')
pylab.plot([p[0] for p in classB],
          [p[1] for p in classB],
          'ro')
pylab.show()

In [9]:
print(data)
print(data[1][2])
    
x_i = []
x_j = []
for sample in data:
    x_i.append(sample[0])
    x_j.append(sample[1])
print(x_i)

[(-0.4817775866423481, -0.676196126111814, -1.0), (-1.3746653598765421, -1.2335653503955357, 1.0), (-0.04952991285713326, -0.477259823574604, -1.0), (-2.435302270702021, 0.5791628345129953, 1.0), (-0.23207345925812842, -0.829241385869637, -1.0), (-1.746084907233614, 0.9006137892797217, 1.0), (-0.3698769551877943, 1.9746639924223914, 1.0), (1.3807418306962262, 0.8160567394627515, 1.0), (0.6481506000006643, -0.2993683817580585, -1.0), (0.006408247677090779, -0.711255503088516, -1.0), (-0.3556075093525846, 0.6450331217049499, -1.0), (0.9314091400315766, -0.7871051525340638, -1.0), (0.86816793401546, -0.5298525064359068, 1.0), (-0.04468918689694391, -0.5181337266696356, -1.0), (1.2636537068237677, 1.1427041812767662, 1.0), (0.09043582719932093, 0.04380816810541566, -1.0), (0.012425436725393221, -0.6643420601684578, -1.0), (-2.0485964643706644, 0.6991611363135597, 1.0), (-2.8139016711910467, 0.17553854976255767, 1.0), (1.6300789364960653, 0.3280724548177084, 1.0)]
1.0
[-0.4817775866423481, 

In [10]:
mP = build_p_matrix(data, kernels.polynomial)
mP = matrix(mP)
# print(fancy_mP)


In [11]:
# q vector is a N long vector with -1's
mQ = matrix([-1.0 for i in range(N)])
# print(mQ)
# h is a vector with all 0's
mH = matrix([0.0 for i in range(N)])
# print(mH)

In [12]:
# the G Matrix is an identity matrix with -1's
mG = (-1)*matrix(numpy.identity(N))
# print(mG)

In [24]:
# Make the call to qp
r = qp(mP, mQ, mG, mH)
a = list(r['x'])
a = [alpha if (alpha > 0.00001) else 0 for alpha in a]
print(a)

# print[(alpha < 0.000005) for alpha in a]

     pcost       dcost       gap    pres   dres
 0: -5.5619e+00 -1.5029e+01  8e+01  7e+00  2e+00
 1: -1.6497e+01 -2.8666e+01  5e+01  5e+00  1e+00
 2: -5.3571e+01 -6.3447e+01  4e+01  3e+00  1e+00
 3: -1.4223e+02 -1.5118e+02  5e+01  3e+00  1e+00
 4: -1.6454e+02 -2.0954e+02  1e+02  2e+00  6e-01
 5: -1.1886e+02 -1.6182e+02  9e+01  6e-01  2e-01
 6: -1.2084e+02 -1.2219e+02  3e+00  2e-02  7e-03
 7: -1.2058e+02 -1.2059e+02  3e-02  2e-04  7e-05
 8: -1.2057e+02 -1.2057e+02  3e-04  2e-06  7e-07
 9: -1.2057e+02 -1.2057e+02  3e-06  2e-08  7e-09
Optimal solution found.
[0, 0.9029756022691927, 0, 0, 0, 0, 1.2421375666813084, 0, 68.02087927035691, 0, 0, 55.59505917601346, 115.38468947143198, 0, 0, 0, 0, 0, 0, 0]


In [39]:
# Pick out the close-to-zero's in the alpha vector
# nonzero_alphas = [alpha for alpha in a if (alpha > 0.0005) else 0 for alpha in a]
nonzero_alphas = [alpha for alpha in a if (alpha > 0.0001)]
for index, sample in enumerate(data):
#     print(sample*a[index])
#     print(a[index])
    nonzero_datapoints = [sample for sample in data if (a[index] > 0)]
nonzero_datapoints = []
for index, sample in enumerate(data):
#     print(index)
    if a[index] > 0:
        nonzero_datapoints.append(sample)
    
#     nonzero_datapoints = [sample if (sample*a[index] > 0) else 0]
print(nonzero_datapoints)
print(nonzero_alphas)
# print(a)
# Save the corresponding X_i's (which has a nonzero alpha)
# x_i = [sample if for sample in data]

[(-1.3746653598765421, -1.2335653503955357, 1.0), (-0.3698769551877943, 1.9746639924223914, 1.0), (0.6481506000006643, -0.2993683817580585, -1.0), (0.9314091400315766, -0.7871051525340638, -1.0), (0.86816793401546, -0.5298525064359068, 1.0)]
[0.9029756022691927, 1.2421375666813084, 68.02087927035691, 55.59505917601346, 115.38468947143198]


In [61]:
#generate new datapoints without a class and feed it into the indicator function which already knows about the non
#zero alphas and corresponding datapoints
new_n = len(nonzero_alphas)
xrange = numpy.arange(-4, 4, 0.05)
yrange = numpy.arange(-4, 4, 0.05)
# for index in range(new_n):
#     new_data = [(xrange[index], yrange[index])]
new_data = []
for i in range(new_n):
    new_data.append((xrange[i], yrange[i]))
# new_data = zip(xrange,yrange)
print(new_data)
# print(xrange)
def indicator(new_data, alphas, support_points, kernel_method):
    for i in range(len(alphas)):
        alphas[i]*support_points[i][2]*kernel_method([support_points[i][0], support_points[i][1]],
                                                     [new_data[i][0], new_data[i][1]])

[ [ i n d i c a t o r ( x , y )
f o r y in yrange ]
f o r x in xrange ]

[(-4.0, -4.0), (-3.9500000000000002, -3.9500000000000002), (-3.9000000000000004, -3.9000000000000004), (-3.8500000000000005, -3.8500000000000005), (-3.8000000000000007, -3.8000000000000007)]


In [63]:
grid = matrix([
        [indicator(new_data[:new_n], nonzero_alphas, nonzero_datapoints, kernels.polynomial) for y in yrange] 
        for x in xrange])


TypeError: invalid type in list